<a href="https://colab.research.google.com/github/M-luthra07/Practice-Deep-Learning/blob/main/MNIST_dataset%2C_improve_autoencoder's_performance_using_convolutional_layers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.linear_model import LogisticRegression
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D, Flatten, Dense, Reshape
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical


In [ ]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train = x_train.astype('float32') / 255.
x_test = x_test.astype('float32') / 255.
x_train = np.reshape(x_train, (len(x_train), 28, 28, 1))
x_test = np.reshape(x_test, (len(x_test), 28, 28, 1))

11490434/11490434 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [ ]:
input_img = Input(shape=(28, 28, 1))

# Encoder
x = Conv2D(32, (3, 3), activation='relu', padding='same')(input_img)
x = MaxPooling2D((2, 2), padding='same')(x)
x = Conv2D(16, (3, 3), activation='relu', padding='same')(x)
x = MaxPooling2D((2, 2), padding='same')(x)
encoded = Conv2D(8, (3, 3), activation='relu', padding='same')(x)
encoded = MaxPooling2D((2, 2), padding='same')(encoded)

In [ ]:
x = Conv2D(8, (3, 3), activation='relu', padding='same')(encoded)
x = UpSampling2D((2, 2))(x)
x = Conv2D(16, (3, 3), activation='relu', padding='same')(x)
x = UpSampling2D((2, 2))(x)
x = Conv2D(32, (3, 3), activation='relu')(x)
x = UpSampling2D((2, 2))(x)
decoded = Conv2D(1, (3, 3), activation='sigmoid', padding='same')(x)

autoencoder = Model(input_img, decoded)
autoencoder.compile(optimizer=Adam(), loss='binary_crossentropy')

# Train the autoencoder
history = autoencoder.fit(x_train, x_train,
                          epochs=20,
                          batch_size=128,
                          shuffle=True,
                          validation_data=(x_test, x_test))

Epoch 1/20
469/469 ━━━━━━━━━━━━━━━━━━━━ 133s 275ms/step - loss: 0.2941 - val_loss: 0.1327
Epoch 2/20
469/469 ━━━━━━━━━━━━━━━━━━━━ 129s 247ms/step - loss: 0.1282 - val_loss: 0.1168
Epoch 3/20
469/469 ━━━━━━━━━━━━━━━━━━━━ 142s 246ms/step - loss: 0.1155 - val_loss: 0.1096
Epoch 4/20
469/469 ━━━━━━━━━━━━━━━━━━━━ 144s 251ms/step - loss: 0.1099 - val_loss: 0.1053
Epoch 5/20
469/469 ━━━━━━━━━━━━━━━━━━━━ 137s 240ms/step - loss: 0.1059 - val_loss: 0.1022
Epoch 6/20
469/469 ━━━━━━━━━━━━━━━━━━━━ 143s 242ms/step - loss: 0.1033 - val_loss: 0.1003
Epoch 7/20
469/469 ━━━━━━━━━━━━━━━━━━━━ 142s 242ms/step - loss: 0.1008 - val_loss: 0.0982
Epoch 8/20
469/469 ━━━━━━━━━━━━━━━━━━━━ 143s 245ms/step - loss: 0.0991 - val_loss: 0.0967
Epoch 9/20
469/469 ━━━━━━━━━━━━━━━━━━━━ 138s 236ms/step - loss: 0.0976 - val_loss: 0.0952
Epoch 10/20
469/469 ━━━━━━━━━━━━━━━━━━━━ 111s 237ms/step - loss: 0.0965 - val_loss: 0.0949
Epoch 11/20
469/469 ━━━━━━━━━━━━━━━━━━━━ 148s 251ms/step - loss: 0.0955 - val_loss: 0.0946
Epoch 12

In [ ]:
encoder = Model(input_img, encoded)

# Flatten encoded representations for classifier
x_train_encoded = encoder.predict(x_train)
x_test_encoded = encoder.predict(x_test)
x_train_encoded = x_train_encoded.reshape((len(x_train_encoded), -1))
x_test_encoded = x_test_encoded.reshape((len(x_test_encoded), -1))

In [ ]:
clf = LogisticRegression(max_iter=1000)
clf.fit(x_train_encoded, y_train)
y_pred = clf.predict(x_test_encoded)


In [ ]:
# Evaluation Metrics
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='macro')
recall = recall_score(y_test, y_pred, average='macro')
f1 = f1_score(y_test, y_pred, average='macro')

print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)

In [ ]:
# Plot training and validation loss
plt.figure(figsize=(12, 5))
plt.subplot(1, 2, 1)
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Autoencoder Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.grid(True)

# Accuracy bar chart
plt.subplot(1, 2, 2)
metrics = ['Accuracy', 'Precision', 'Recall', 'F1 Score']
values = [accuracy, precision, recall, f1]
plt.bar(metrics, values, color=['blue', 'green', 'orange', 'red'])
plt.ylim(0, 1)
plt.title('Classification Performance')
plt.ylabel('Score')
plt.grid(True, axis='y')

plt.tight_layout()
plt.show()
